## 1. Setting Up the Environment

Before we dive into building our agent, we need to set up the necessary environment. This involves installing required packages and ensuring our Python environment is ready for development.

In [70]:
# Install termcolor for colored terminal outputs
%pip install termcolor langgraph

Note: you may need to restart the kernel to use updated packages.


In [71]:
# Import necessary libraries
from langgraph.graph.message import add_messages
from typing import TypedDict, Annotated
from datetime import datetime, timezone
from termcolor import colored
import time

In [72]:
# Define the state object for the agent graph
class AgentGraphState(TypedDict):
    start_chain: Annotated[list, add_messages]
    transcription_response: Annotated[list, add_messages]
    portuguese_response: Annotated[list, add_messages]
    spanish_response: Annotated[list, add_messages]
    end_chain: Annotated[list, add_messages]

# Function to retrieve specific parts of the agent state
def get_agent_graph_state(state: AgentGraphState, state_key: str):

    if state_key == "transcription_all":
        return state["transcription_response"]
    elif state_key == "transcription_latest":
        return (
            state["transcription_response"][-1]
            if state["transcription_response"]
            else []
        )

    if state_key == "portuguese_all":
        return state["portuguese_response"]
    elif state_key == "portuguese_latest":
        return state["portuguese_response"][-1] if state["portuguese_response"] else []

    if state_key == "spanish_all":
        return state["spanish_response"]
    elif state_key == "spanish_latest":
        return state["spanish_response"][-1] if state["spanish_response"] else []

    else:
        return None


# Initial state setup
state = {
    "start_chain": [],
    "transcription_response": [],
    "portuguese_response": [],
    "spanish_response": [],
    "end_chain": [],
}

In [73]:
import asyncio

class TranscriptAgent:

    def __init__(self, state: AgentGraphState, words: list):
        self.state = state
        self.words = words

    async def invoke(self):
        for word in self.words:
            await asyncio.sleep(5)  # Simulate async operation
            print(colored(f"TRANSCRIPT_AGENT 📝: Processing word '{word}'", "cyan"))
            self.state["transcription_response"].append(word)
            yield self.state

In [74]:
from langgraph.graph import START, END, StateGraph

def create_graph() -> StateGraph:
    graph = StateGraph(AgentGraphState)

    async def text_node_function(state):
        agent = TranscriptAgent(
            state=state, words=["hello", "world", "this", "is", "a", "test"]
        )
        async for new_state in agent.invoke():
            yield new_state

    graph.add_node("text_node", text_node_function)

    # Define the flow of the graph
    graph.add_edge(START, "text_node")
    graph.add_edge("text_node", END)

    return graph

def compile_workflow(graph: StateGraph):
    workflow = graph.compile()
    return workflow

In [75]:
# Create the graph and compile the workflow
graph = create_graph()
workflow = compile_workflow(graph)
print("Graph and workflow created.")

# Define workflow parameters
iterations = 10
limit = {"recursion_limit": iterations}
verbose = True
dict_inputs = {"start_chain": "start"}

# Execute the workflow and print state changes
async for event in workflow.astream_events(dict_inputs, version="v1"):
    if verbose:
        print(colored(event,"magenta"))
    else:
        print("\n")

Graph and workflow created.
{'event': 'on_chain_start', 'run_id': 'e04956fd-c97a-4141-86d7-6ee425f590d5', 'name': 'LangGraph', 'tags': [], 'metadata': {}, 'data': {'input': {'start_chain': 'start'}}, 'parent_ids': []}
{'event': 'on_chain_start', 'name': '__start__', 'run_id': '84b6259e-cfae-4723-9aec-474bf485678d', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef50c1e-82cf-683a-bffe-e98172474940'}, 'data': {'input': {'start_chain': 'start'}}, 'parent_ids': []}
{'event': 'on_chain_end', 'name': '__start__', 'run_id': '84b6259e-cfae-4723-9aec-474bf485678d', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef50c1e-82cf-683a-bffe-e98172474940'}, 'data': {'input': {'start_chain': 'start'}, 'output': {'start_chain':